<h3>Tests for RNAfold</h3>

In [1]:
import pandas as pd

In [17]:
aptabase = pd.read_csv('Aptamers_2D.csv')
aptabase.head()

,PDB_ID,Sequence,2D,Len,Structure,RNAfold_ref
0,1JVE,cctaattataacgaagttataattagg,(((((((((((((.))))))))))))),27,Triplex-DNA,1.0
1,1NGU,ctctccttgtatttcttacaaaaagag,(((((((((((.....))))))))))),27,Hairpin with pseudoknots,1.0
2,2OEY,ccatcgtctacctttggtaggatgg,(((((..(((((...)))))))))),25,Hairpin with loops,1.0
3,2VWJ,aauggagacacggcttttgccgtgtc,......((((((((....)))))))),26,Hairpin with dangling ends,1.0
4,3THW,cctctatctgaagccgatcgatgaagcatcgatcgcacagcttcag...,((((((((((((((((((((((((.))))))))))....)))))))...,53,Hairpin with loops,1.0


In [3]:
len(aptabase)

69

<h3>RNAfold</h3>


In [4]:
!RNAfold -g -v test.fa

Processing 1. input file "test.fa"
>1JVE
ccuaauuauaacgaaguuauaauuagg
((((((((((((...)))))))))))) (-11.00)
>1NGU
cucuccuuguauuucuuacaaaaagag
((((..(((((.....)))))..)))) ( -6.40)
>230D
gggguuuugggguuuugggguuuugggg
++++....++++....++++....++++ (-26.37)


In [5]:
!RNAfold -h

Usage: RNAfold [OPTIONS] [<input0.fa>] [<input1.fa>]...
Calculate minimum free energy secondary structures and partition function of
RNAs

The program reads RNA sequences, calculates their minimum free energy (mfe)
structure and prints the mfe structure in bracket notation and its free energy.
If not specified differently using commandline arguments, input is accepted
from stdin or read from an input file, and output printed to stdout. If the -p
option was given it also computes the partition function (pf) and base pairing
probability matrix, and prints the free energy of the thermodynamic ensemble,
the frequency of the mfe structure in the ensemble, and the ensemble diversity
to stdout.

It also produces PostScript files with plots of the resulting secondary
structure graph and a "dot plot" of the base pairing matrix.
The dot plot shows a matrix of squares with area proportional to the pairing
probability in the upper right half, and one square for each pair in the
minimum free energy

In [6]:
!rm *ps
!rm *fasta

In [7]:
def aptamers_to_fasta(db, filename):

    with open(filename, 'w') as fasta:

        ids = list(db['PDB_ID'])
        seqs = list(db['Sequence'])

        for i in range(len(ids)):
            print('>'+ids[i], file=fasta)
            print(seqs[i], file=fasta)

aptamers_to_fasta(aptabase, 'Aptamers_2D.fasta')


In [8]:
!RNAfold -g -v Aptamers_2D.fasta --outfile=rnafold_predictions.fasta

Processing 1. input file "Aptamers_2D.fasta"


In [9]:
def fastaparser(filename):

    with open(filename, 'r') as fasta:
        lines = fasta.readlines()
        preds = []
        for i in range(2, len(lines), 3):
            pred = lines[i].split()
            preds.append(pred[0])

    return preds

rnafold_preds = fastaparser('rnafold_predictions.fasta')
len(rnafold_preds)



69

In [26]:
def accuracy_2D(pred, ref):

    count = 0 
    for k in range(len(pred)):

        if pred[k] == ref[k]:
            count += 1

        elif (pred[k]=='+') and (ref[k] in ('(', ')', '[', ']', '{', '}')):
            count += 1

        elif (pred[k] in ('[', '{')) and (ref[k] in ('[', '{')):
            count +=1

        elif (pred[k] in (']', '}')) and (ref[k] in (']', '}')):
            count +=1

    return round(count/len(pred), 2)

accuracy_2D(rnafold_preds[10], list(aptabase['2D'])[10])




0.96

In [27]:
aptabase.head(15)

,PDB_ID,Sequence,2D,Len,Structure,RNAfold_ref,RNAfold_2D,RNAfold_real
0,1JVE,cctaattataacgaagttataattagg,(((((((((((((.))))))))))))),27,Triplex-DNA,1.00,((((((((((((...)))))))))))),0.93
1,1NGU,ctctccttgtatttcttacaaaaagag,(((((((((((.....))))))))))),27,Hairpin with pseudoknots,1.00,((((..(((((.....)))))..)))),0.85
2,2OEY,ccatcgtctacctttggtaggatgg,(((((..(((((...)))))))))),25,Hairpin with loops,1.00,(((((..(((((...)))))))))),1.00
3,2VWJ,aauggagacacggcttttgccgtgtc,......((((((((....)))))))),26,Hairpin with dangling ends,1.00,......((((((((....)))))))),1.00
4,3THW,cctctatctgaagccgatcgatgaagcatcgatcgcacagcttcag...,((((((((((((((((((((((((.))))))))))....)))))))...,53,Hairpin with loops,1.00,(((((((((((((((((((((((...)))))))))....)))))))...,0.96
5,4HT4,cgcgaacggaacgttcgcataagtgcgc,.((((((((.))))))))........(),28,Hairpin with dangling ends,1.00,.(((((((...)))))))..........,0.86
6,1NGO,ctctttttgtaagaaatacaaggagag,(((((((((((.....))))))))))),27,Hairpin,1.00,(((((((((((.....))))))))))),1.00
7,5N2Q,actttatgaaaataaagtatagtgtg,(((((((....)))))))........,26,Hairpin with dangling ends,1.00,(((((((....)))))))........,1.00
8,3H25,cctttccccctacccgaagggtggggg,......(((((((((...))))))))),27,Hairpin with dangling ends,1.00,......(((((((((...))))))))),1.00
9,1AW4,acctgggggagtattgcggaggaaggtaaacctgggggagtattgc...,(((((()([(((...))))..{)))))]}(((((()([(((...))...,27,Hairpin with pseudoknots,1.00,((((...(.((((((.(..(((........)))...).)))))).)...,0.26


In [12]:
rnafold_accuracy_points = []
ref2d = list(aptabase['2D'])

for i in range(len(ref2d)):
    rnafold_accuracy_points.append(accuracy_2D(rnafold_preds[i], ref2d[i]))



In [13]:
#% of accurate structures
acc_counter = 0
for ac in rnafold_accuracy_points:
    if ac >= 0.85:
        acc_counter +=1

print('% of Accurate structures (accuracy >= 0.85) =', 100*(acc_counter/len(rnafold_accuracy_points)))

% of Accurate structures (accuracy >= 0.85) = 49.275362318840585


In [14]:
#Full accuracy
nt_accuracy = accuracy_2D(''.join(rnafold_preds), ''.join(ref2d))
print('Nucleotide Accuracy=', nt_accuracy)

Nucleotide Accuracy= 0.76


If we take into account non-canonical base pairs, % of accurate structures and average accuracy of RNAfold on dataset of short DNA aptamers is significantly lower than claimed in paper (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6975895/):

Claimed % of accurate structures = 64%

Real % of accurate structures = 49%

Claimed nucleotide accuracy = 0.84

Real nucleotide accuracy = 0.76

In [18]:
aptabase['RNAfold_2D'] = pd.Series(rnafold_preds)
aptabase['RNAfold_real'] = pd.Series(rnafold_accuracy_points)
aptabase

,PDB_ID,Sequence,2D,Len,Structure,RNAfold_ref,RNAfold_2D,RNAfold_real
0,1JVE,cctaattataacgaagttataattagg,(((((((((((((.))))))))))))),27,Triplex-DNA,1.00,((((((((((((...)))))))))))),0.93
1,1NGU,ctctccttgtatttcttacaaaaagag,(((((((((((.....))))))))))),27,Hairpin with pseudoknots,1.00,((((..(((((.....)))))..)))),0.85
2,2OEY,ccatcgtctacctttggtaggatgg,(((((..(((((...)))))))))),25,Hairpin with loops,1.00,(((((..(((((...)))))))))),1.00
3,2VWJ,aauggagacacggcttttgccgtgtc,......((((((((....)))))))),26,Hairpin with dangling ends,1.00,......((((((((....)))))))),1.00
4,3THW,cctctatctgaagccgatcgatgaagcatcgatcgcacagcttcag...,((((((((((((((((((((((((.))))))))))....)))))))...,53,Hairpin with loops,1.00,(((((((((((((((((((((((...)))))))))....)))))))...,0.96
...,...,...,...,...,...,...,...,...
64,4U5M,tggtggtggtggttgtggtggtggtgtt,(([.)])([.)]..(.[).](.[).]..,28,G-quadruplex,0.71,.++.++.++.++................,0.64
65,2N3M,tggtggtggttgttgtggtggtggtggt,(([.)].[{.)]..}.([.)].([.)].,28,G-quadruplex,0.71,................++.++.++.++.,0.64
66,201D,ggggttttggggttttggggttttgggg,(((((.)[))))....((((...])))),28,G-quadruplex,1.00,++++....++++....++++....++++,0.86
67,5J6U,ggggtttggggttttggggaagggg,((((.[.))))....(((]...))),25,G-quadruplex,1.00,++++...++++....++++..++++,0.92


In [19]:
aptabase.to_csv('RNAfold_benchmarking.csv')

In [31]:
def TN(pred, ref):

    counter = 0
    for i in range(len(ref)):
        if (pred[i] == '.') and (ref[i]=='.'):
            counter +=1

    return counter

def FN(pred, ref):

    counter = 0
    for i in range(len(ref)):
        if (pred[i] == '.') and (ref[i]!='.'):
            counter +=1

    return counter


   


In [28]:
def find_basepairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '(':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ')':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return result

def find_knotpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '[':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == ']':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return result

def find_otherpairs(s):
    stack = []
    #result = {}
    result = []

    for index, char in enumerate(s):
        if char == '{':
            # Запоминаем индекс открывающей скобки
            stack.append(index)
        elif char == '}':
            # Если нашли закрывающую скобку, сопоставляем с последней открывающей
            if stack:
                open_index = stack.pop()
                #result[open_index] = index
                result.append((open_index, index))

    return result

def TP(pred,ref):

    counter=0

    pred_basepairs = find_basepairs(pred)
    pred_knotpairs = find_knotpairs(pred)
    pred_otherpairs = find_otherpairs(pred)

    ref_basepairs = find_basepairs(ref)
    ref_knotpairs = find_knotpairs(ref)
    ref_otherpairs = find_otherpairs(ref)

    for b in pred_basepairs:
        if b in ref_basepairs:
            counter += 2

    for k in pred_knotpairs:
        if k in ref_knotpairs:
            counter += 2

    for o in pred_otherpairs:
        if k in ref_otherpairs:
            counter += 2

    return counter

    



In [32]:
tp = TP(''.join(rnafold_preds), ''.join(ref2d))
fn = FN(''.join(rnafold_preds), ''.join(ref2d))
tn = TN(''.join(rnafold_preds), ''.join(ref2d))
fp = len(''.join(rnafold_preds)) - (tp+fn+tn)

Precision = tp/(tp+fp)
Recall = tp/(tp+fn)
F1 = 2*Precision*Recall/(Precision+Recall)

print('Precision =', round(Precision, 2))
print('Recall =', round(Recall, 2))
print('F1-score =', round(F1, 2))

Precision = 0.67
Recall = 0.7
F1-score = 0.68
